In [1]:
import os
import fitz
from tqdm.auto import tqdm

pdf_path = "U:\\LEGALSENSE\\script\\Pakistan Penal Code.pdf"


def text_formatter(text: str) -> str:
    cleaned_text = text.replace("\n", "").strip()
    return cleaned_text


def open_and_read_pdf(pdf_path: str) -> list[dict]:
    doc = fitz.open(pdf_path)
    pages_and_text = []
    for page_number, page in tqdm(enumerate(doc)):
        text = page.get_text()
        text = text_formatter(text=text)
        pages_and_text.append({"page_number": page_number - 4,
                               "page_char_count": len(text),
                               "page_word_count": len(text.split(" ")),
                               "page_sentence_count": len(text.split(". ")),
                               "page_token_count": len(text) / 4,
                               "text": text})
    return pages_and_text


pages_and_text = open_and_read_pdf(pdf_path=pdf_path)
pages_and_text[:2]

0it [00:00, ?it/s]

[{'page_number': -4,
  'page_char_count': 1651,
  'page_word_count': 283,
  'page_sentence_count': 14,
  'page_token_count': 412.75,
  'text': 'Pakistan Penal Code (Act XLV of 1860) Act XLV of 1860 October 6th, 1860 Amended by: Protection of Women (Criminal Laws Amendment) Act, 2006,Criminal Laws (Amendment) Act, 2004 (I of 2005),Criminal Law (Amendment) Ordinance (LXXXV of 2002),Criminal Laws (Reforms) Ordinance (LXXXVI of 2002),etc.  Whereas it is expedient to provide a general Penal Code for Pakistan:  It is enacted as follows:-   CHAPTER I INTRODUCTION 1. Title and extent of operation of the Code. This Act shall be called the Pakistan Penal Code, and shall take effect throughout Pakistan.   2. Punishment of offences committed within Pakistan. Every person shall be liable to punishment under this Code and not otherwise for every act or omission contrary to the provisions thereof, of which he shall be guilty within Pakistan.   3. Punishment of offences committed beyond, but which by 

In [21]:
import nltk
nltk.download('punkt')

from nltk.tokenize import sent_tokenize

text = "My name is Huzaifa. I am a CS student at PU"
sentences = sent_tokenize(text)

print(sentences)



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Huzaifa\AppData\Roaming\nltk_data...


['My name is Huzaifa.', 'I am a CS student at PU']


[nltk_data]   Package punkt is already up-to-date!


In [22]:
for item in tqdm(pages_and_text):
    item["sentences"] = list(sent_tokenize(item["text"]))
    item["page_sentence_count"] = len(item["sentences"])
    

  0%|          | 0/164 [00:00<?, ?it/s]

In [25]:
import random

random.sample(pages_and_text, k=1)

[{'page_number': 102,
  'page_char_count': 2324,
  'page_word_count': 404,
  'page_sentence_count': 17,
  'page_token_count': 581.0,
  'text': 'A. Whoever assaults or uses criminal force to any woman and strips her of her clothes and in that condition, exposes her to the public view, shall be punished with death or with imprisonment for life, and shall also be liable to fine.   ] 137  355. Assault or criminal force with intent to dishonour person, otherwise than on grave provocation: Whoever assaults or uses criminal force to any person, intending thereby to dishonour that person, otherwise than on grave and sudden provocation given by that person, shall be punished with imprisonment of either description for a term which may extend to two years, or with fine, or with both.   356. Assault or criminal force in attempt to commit theft of property carried by a person: Whoever assaults or uses criminal force to any person in attempting to commit theft on any property which that person is t

In [26]:
chunk_size_num = 10

def split_list(input_list : list[str],
               slice_size : int = chunk_size_num) -> list[list[str]]:
    return [input_list[i:i + slice_size] for i in range(0, len(input_list), slice_size)]

test_split = list(range(39))
split_list(test_split)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
 [30, 31, 32, 33, 34, 35, 36, 37, 38]]

In [27]:
for item in tqdm(pages_and_text):
    item["sentence_chunks"] = split_list(input_list=item["sentences"],
                                         slice_size=chunk_size_num)
    item['num_chunks'] = len(item["sentence_chunks"])

  0%|          | 0/164 [00:00<?, ?it/s]

In [28]:
random.sample(pages_and_text, k=1)

[{'page_number': 14,
  'page_char_count': 2788,
  'page_word_count': 532,
  'page_sentence_count': 20,
  'page_token_count': 697.0,
  'text': "likely to cause, to the person giving the consent or on whose behalf the consent is given.  Illustration Causing miscarriage (unless caused in good faith for the purpose of saving the life of the woman) to an offence independently of any harm which it may cause or be intended, to cause to the woman. Therefore it is not an offence by reason of such harm; and the consent of the woman or of her guardian to the causing of such miscarriage dose not justify the act.    92. Act done in good faith for benefit of a person without consent: Nothing an offence by reason of any harm which it may cause to a person by whose benefit it is done in good faith even without that person's consent, if the Circumstances are such that is impossible for that person to signify consent, or if that person is incapable of giving consent, and has no guardian or other person 

In [29]:
import pandas as pd

df = pd.DataFrame(pages_and_text)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count,page_token_count,num_chunks
count,164.00,164.00,164.00,164.00,164.00,164.0
mean,77.50,2462.57,447.39,14.12,615.64,1.9
std,47.49,449.37,75.68,4.24,112.34,0.5
min,-4.00,1387.00,268.00,6.00,346.75,1.0
25%,36.75,2198.00,402.75,11.00,549.50,2.0
50%,77.50,2548.50,455.50,14.00,637.12,2.0
75%,118.25,2783.50,504.25,17.00,695.88,2.0
max,159.00,3370.00,605.00,27.00,842.50,3.0


In [38]:
import re

pages_and_chunks = []

for item in tqdm(pages_and_text):
    for sentence_chunk in item["sentence_chunks"]:
        chunk_dict = {}
        chunk_dict["page_number"] = item["page_number"]

        joined_sentence_chunk = "".join(sentence_chunk).replace(" "," ").strip()

        chunk_dict["sentence_chunk"] = joined_sentence_chunk

        chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
        chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / 4

        pages_and_chunks.append(chunk_dict)

len(pages_and_chunks)

  0%|          | 0/164 [00:00<?, ?it/s]

312

In [40]:
random.sample(pages_and_chunks, k=1)

[{'page_number': 101,
  'sentence_chunk': 'Assault or criminal force to deter public servant from discharge of his duty: Whoever assaults or uses criminal force to any person being a public servant in the execution of his duty as such public servant, or with intent to prevent or deter that person from discharging his duty as such public servant, or in consequence of anything done or attempted to be done by such person in the lawful discharge of his duty as such public servant, shall be punished with imprisonment of either description for a term which may extend to two years, or with fine or with both.354.Assault or criminal force to woman with intent to outrage her modesty: Whoever assaults or uses criminal force to any woman, intending to outrage or knowing it to be likely that he will thereby outrage her modesty, shall be punished with imprisonment of either description for a term which may extend to two years or with fine, or with both.137[  354- Assault or use of criminal force to 

In [42]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-mpnet-base-v2', device='cpu')

%% time

model.to('cpu')

for item in tqdm(pages_and_chunks):
    item["embedding"] = model.encode(item["sentence_chunk"])



modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

UsageError: Line magic function `%%` not found.


In [48]:
text_chunks = [item["sentence_chunk"] for item in pages_and_chunks]
text_chunks[220]

'394.Voluntarily causing hurt in committing robbery: If any person, in committing or in attempting to commit robbery, voluntarily causes hurt, such person, and any other person jointly concerned in committing or attempting to commit such robbery, shall be punished with imprisonment for life, or with rigorous imprisonment for a term  157[which shall not be less than four years nor more than] 157 ten years, and shall also be liable to fine.395.Punishment for dacoity: Whoever commits dacoity shall be punished with imprisonment for life, or with rigorous imprisonment for a term which shall not be less than four years nor more than ten years and shall also be liable to fine.396.Dacoity with murder: If any one of five or more persons, who are conjointly committing dacoity, commits murder in so committing dacoity, everyone of those persons shall be punished with death, or imprisonment for life, or rigorous imprisonment for a term which  158[shall not be less than four years nor more than] 158

In [49]:
len(text_chunks)

312

In [50]:
text_chunk_embeddings = model.encode(text_chunks, batch_size=32, convert_to_tensor=True)

text_chunk_embeddings

tensor([[ 2.4755e-02, -6.1537e-02,  1.9320e-02,  ...,  4.5774e-02,
         -4.1376e-02,  2.0377e-02],
        [ 1.0911e-02, -4.6941e-02,  1.6341e-02,  ...,  4.4782e-02,
         -8.5377e-02, -5.3400e-03],
        [ 2.6155e-02, -1.9259e-02,  2.7876e-02,  ...,  7.4784e-02,
         -7.1943e-02,  2.9807e-02],
        ...,
        [-1.5212e-03,  1.3224e-02,  2.2889e-02,  ..., -1.7561e-02,
         -9.9031e-02,  2.0125e-02],
        [ 1.4144e-02,  1.3743e-02, -7.9148e-05,  ..., -1.4128e-04,
         -7.8975e-02, -7.6367e-03],
        [-1.5661e-02, -5.7370e-02,  9.2282e-03,  ...,  5.9224e-02,
         -3.1768e-02,  3.1754e-02]])